In [ ]:
import cv2
import boto3
import botocore
from datetime import datetime
import sys
from dotenv import dotenv_values
import json


In [ ]:
config = dotenv_values(".env")


In [ ]:
s3 = boto3.client("s3")
reko = boto3.client("rekognition")


In [ ]:
def variance_of_laplacian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()


In [ ]:
cam = cv2.VideoCapture(0)

print("Starting camera...")

cam.set(cv2.CAP_PROP_AUTOFOCUS, 0)
cam.set(cv2.CAP_PROP_FOCUS, 360)
cam.set(cv2.CAP_PROP_BRIGHTNESS, 130)
cam.set(cv2.CAP_PROP_SHARPNESS, 125)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

pic_taken = False
first_loop = True

while (True):
    if first_loop:
        print("Taking pictures...")
        print("Try to stay still...")
        first_loop = False
    result, image = cam.read()
    if result:
        var = variance_of_laplacian(image)
        if var > 30:
            capture_filename = datetime.now().strftime(f"%d-%m-%Y_%H-%M-%S.jpg")
            cv2.imwrite(f"./captured_images/{capture_filename}", image)
            pic_taken = True
            print("Picture taken...")
            break
        else:
            continue

cv2.destroyAllWindows()
cam.release()


In [ ]:
bucket = config["bucket_name"]
base_face = config["base_face_file"]


In [ ]:
def upload_image(bucket, file):
    is_uploaded = False
    if pic_taken:
        with open(f"./captured_images/{capture_filename}", "rb") as data:
            try:
                s3.upload_fileobj(
                    data,
                    bucket,
                    file
                )
                is_uploaded = True
            except botocore.exceptions.ClientError as error:
                print(error)
    return is_uploaded


In [ ]:
def delete_image(bucket, file):
    is_deleted = False
    try:
        s3.delete_object(
            Bucket=bucket,
            Key=file
        )
        is_deleted = True
    except botocore.exceptions.ClientError as error:
        print(error)
    return is_deleted


In [ ]:
def compare_face(bucket, file, base_face, threshold=85):
    try:
        response = reko.compare_faces(
            SourceImage={
                "S3Object": {
                    "Bucket": bucket,
                    "Name": base_face
                }
            },
            TargetImage={
                "S3Object": {
                    "Bucket": bucket,
                    "Name": file
                }
            },
            SimilarityThreshold=threshold,
        )
    except botocore.exceptions.ClientError as error:
        print(error)

    result = json.loads(json.dumps(response))

    print(f"Source Face Confidence: {result['SourceImageFace']['Confidence']}")


In [ ]:
upload_image(bucket, capture_filename)
compare_face(bucket, base_face, capture_filename)
delete_image(bucket, capture_filename)
